In [1]:
import numpy as np
from gym import spaces
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from typing import Tuple
from pathlib import Path
import random
from recsim import document
from recsim import user
from recsim.choice_model import MultinomialLogitChoiceModel
from recsim.simulator import environment
from recsim.simulator import recsim_gym

2022-11-02 14:47:37.988370: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-02 14:47:38.109222: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /raid/home/aayush/git/rl_recsys/.venv/lib/python3.9/site-packages/cv2/../../lib64:
2022-11-02 14:47:38.109254: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-02 14:47:38.139233: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when

In [2]:
DATASET_NAME="Spotify"
_DATA_PATH = Path(Path.home() / os.environ.get("DATA_PATH"))
_DATASET_PATH = _DATA_PATH / DATASET_NAME
songs=pd.read_feather(_DATASET_PATH)

In [3]:
songs

,year,name,artists,popularity,valence,id,danceability,loudness,speechiness,acousticness,liveness,label
0,1921,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...","['Sergei Rachmaninoff', 'James Levine', 'Berli...",4,0.0594,4BJqT0PrAfrxzMOxytFOIz,0.279,0.624916,0.0366,0.98200,0.6650,0
1,1921,Clancy Lowered the Boom,['Dennis Day'],5,0.9630,7xPhfUan2yNtyFG0cUWkt8,0.819,0.744797,0.4150,0.73200,0.1600,2
2,1921,Gati Bali,['KHP Kridhamardawa Karaton Ngayogyakarta Hadi...,5,0.0394,1o6I8BglA6ylDMrIELygv1,0.328,0.707071,0.0339,0.96100,0.1010,0
3,1921,Danny Boy,['Frank Parker'],3,0.1650,3ftBPsC5vPBKxYSee08FDH,0.275,0.793736,0.0354,0.96700,0.3810,0
4,1921,When Irish Eyes Are Smiling,['Phil Regan'],2,0.2530,4d6HGyGT8e121BsdKmw9v6,0.418,0.781521,0.0380,0.95700,0.2290,0
...,...,...,...,...,...,...,...,...,...,...,...,...
170648,2020,China,"['Anuel AA', 'Daddy Yankee', 'KAROL G', 'Ozuna...",72,0.6080,0KkIkfsLEJbrcIhYsCL7L5,0.786,0.881654,0.0881,0.08460,0.0822,3
170649,2020,Halloweenie III: Seven Days,['Ashnikko'],68,0.7340,0OStKKAuXlxA0fMH54Qs6E,0.717,0.845353,0.0605,0.20600,0.1010,3
170650,2020,AYA,['MAMAMOO'],76,0.6370,4BZXVFYCb76Q0Klojq4piV,0.634,0.904769,0.0809,0.10100,0.2580,3
170651,2020,Darkness,['Eminem'],70,0.1950,5SiZJoLXp3WOl3J4C8IK0d,0.671,0.827484,0.3080,0.00998,0.6430,3


In [4]:
class LTSUserState(user.AbstractUserState):
  def __init__(self, age, gender, valence, danceability, loudness, speechiness, acousticness, liveness, mood):
    ## Transition model parameters
    ## State variables
    ##############################
    self.age = age
    self.gender=gender
    self.valence = valence
    self.danceability = danceability

    ## Engagement parameters
    self.loudness = loudness
    self.speechiness = speechiness
    self.acousticness = acousticness
    self.liveness = liveness

    
    self.mood=mood

    # Noise
    #self._observation_noise = observation_noise_stddev

  def create_observation(self):
    """User's state is not observable."""
    clip_low, clip_high = (-1.0 / (1.0 * self._observation_noise),
                           1.0 / (1.0 * self._observation_noise))
    noise = stats.truncnorm(
        clip_low, clip_high, loc=0.0, scale=self._observation_noise).rvs()
    noisy_sat = self.satisfaction + noise
    return np.array([noisy_sat,])

  @staticmethod
  def observation_space():
    return spaces.Box(shape=(1,), dtype=np.float32, low=-2.0, high=2.0)
  
  # scoring function for use in the choice model -- the user is more likely to
  # click on more chocolatey content.
  def score_document(self, doc_obs):
    return 1 - doc_obs

In [5]:
class LTSStaticUserSampler(user.AbstractUserSampler):
  _state_parameters = None

  def __init__(self,
               user_ctor=LTSUserState,
              #  age=np.random.random_integers(20,60),
              #  gender=np.random.random_integers(1),
              #  valence=np.random.uniform(0.0,1.0),
              #  danceability=np.random.normal(0.5373955347986852,0.17613721955546152),
              #  loudness=np.random.gumbel(0.8004193058243345,0.0690033070151354),
              #  speechiness=np.random.laplace(0.045,0.06335336735949558),
              #  acousticness=np.random.uniform(0.0,0.996),
              #  liveness=np.random.laplace(0.136,0.11092283130094402),
              #  mood=np.random.random_integers(3),
               **kwargs):
      self._state_parameters = {}#'age' : age,
    #                           'gender_group' : gender_group,
    #                           'valence': valence,
    #                           'danceability': danceability,
    #                           'loudness': loudness,
    #                           'speechiness': speechiness,
    #                           'acousticness': acousticness,
    #                           'liveness': liveness,
    #                           'mood': mood
                             
      super(LTSStaticUserSampler, self).__init__(user_ctor, **kwargs)

  def sample_user(self):
    self._state_parameters['age']=np.random.random_integers(20,60)
    self._state_parameters['gender']= np.random.random_integers(2)
    self._state_parameters['valence']= np.random.uniform(0.0,1.0)
    self._state_parameters['danceability']= np.random.normal(0.5373955347986852,0.17613721955546152)
    self._state_parameters['loudness']= np.random.gumbel(0.8004193058243345,0.0690033070151354)
    self._state_parameters['speechiness']= np.random.laplace(0.045,0.06335336735949558)
    self._state_parameters['acousticness']= np.random.uniform(0.0,0.996)
    self._state_parameters['liveness']= np.random.laplace(0.136,0.11092283130094402)
    self._state_parameters['mood']= np.random.random_integers(4)
    
    # starting_nke = ((self._rng.random_sample() - .5) *
    #                 (1 / (1.0 - self._state_parameters['memory_discount'])))
    # self._state_parameters['net_genre_exposure'] = starting_nke
    return self._user_ctor(**self._state_parameters)

In [6]:
sampler = LTSStaticUserSampler()
#starting_nke = []

for i in range(1000):
  sampled_user = sampler.sample_user()
  #starting_nke.append(sampled_user.net_genre_exposure)
print(sampled_user.age)
print(dir(sampled_user))
for att in dir(sampled_user):
    print (att, getattr(sampled_user,att))

20
['NUM_FEATURES', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', 'acousticness', 'age', 'create_observation', 'danceability', 'gender', 'liveness', 'loudness', 'mood', 'observation_space', 'score_document', 'speechiness', 'valence']
NUM_FEATURES None
__abstractmethods__ frozenset()
__class__ <class '__main__.LTSUserState'>
__delattr__ <method-wrapper '__delattr__' of LTSUserState object at 0x7f2d981b6f10>
__dict__ {'age': 20, 'gender': 1, 'valence': 0.2627157328047033, 'danceability': 0.5719776169537104, 'loudness': 0.8208226222861714, 'speechiness': -0.04577651013445809, 'acousticness': 0.46769525798866896, 'liveness': 0.09771716210457732, 'mood': 2}
__dir

/tmp/ipykernel_1692894/1202549466.py:29: DeprecationWarning: This function is deprecated. Please call randint(20, 60 + 1) instead
  self._state_parameters['age']=np.random.random_integers(20,60)
/tmp/ipykernel_1692894/1202549466.py:30: DeprecationWarning: This function is deprecated. Please call randint(1, 2 + 1) instead
  self._state_parameters['gender']= np.random.random_integers(2)
/tmp/ipykernel_1692894/1202549466.py:37: DeprecationWarning: This function is deprecated. Please call randint(1, 4 + 1) instead
  self._state_parameters['mood']= np.random.random_integers(4)


In [7]:
sampler = LTSStaticUserSampler()
User = pd.DataFrame(columns=['User_ID', 'age', 'gender','valence','danceability','loudness','speechiness','acousticness','liveness','mood'])
#starting_nke = []

for i in range(1000):
    sampled_user = sampler.sample_user()
    k=sampled_user
    User=User.append({'User_ID':i, 'age': k.age , 'gender':k.gender,'valence':k.valence,'danceability':k.danceability,'loudness':k.loudness,'speechiness':k.speechiness,'acousticness':k.acousticness,'liveness':k.liveness,'mood':k.mood}, ignore_index=True)

/tmp/ipykernel_1692894/1202549466.py:29: DeprecationWarning: This function is deprecated. Please call randint(20, 60 + 1) instead
  self._state_parameters['age']=np.random.random_integers(20,60)
/tmp/ipykernel_1692894/1202549466.py:30: DeprecationWarning: This function is deprecated. Please call randint(1, 2 + 1) instead
  self._state_parameters['gender']= np.random.random_integers(2)
/tmp/ipykernel_1692894/1202549466.py:37: DeprecationWarning: This function is deprecated. Please call randint(1, 4 + 1) instead
  self._state_parameters['mood']= np.random.random_integers(4)
/tmp/ipykernel_1692894/452295700.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  User=User.append({'User_ID':i, 'age': k.age , 'gender':k.gender,'valence':k.valence,'danceability':k.danceability,'loudness':k.loudness,'speechiness':k.speechiness,'acousticness':k.acousticness,'liveness':k.liveness,'mood':k.mood}, ignore_index=Tr

In [8]:
User[User["mood"]==4]

,User_ID,age,gender,valence,danceability,loudness,speechiness,acousticness,liveness,mood
1,1.0,49.0,1.0,0.375064,0.470769,0.866105,-0.072958,0.019419,0.184157,4.0
3,3.0,31.0,1.0,0.990559,0.637021,0.893599,0.064175,0.027529,0.130927,4.0
4,4.0,29.0,2.0,0.845593,0.242171,0.971975,0.069892,0.156152,-0.024053,4.0
8,8.0,29.0,1.0,0.486567,0.245875,0.790789,0.064106,0.181303,0.185919,4.0
17,17.0,48.0,2.0,0.847029,0.499064,0.845032,0.076124,0.813384,-0.126152,4.0
...,...,...,...,...,...,...,...,...,...,...
983,983.0,21.0,1.0,0.992835,0.548403,1.002648,0.057948,0.810297,0.147846,4.0
985,985.0,36.0,2.0,0.928105,0.345723,0.930257,0.014865,0.675192,0.227889,4.0
987,987.0,26.0,2.0,0.652141,0.610195,0.890367,-0.108803,0.743271,0.294046,4.0
993,993.0,41.0,1.0,0.558799,0.502032,1.020575,0.014091,0.441379,0.091055,4.0
